## 1.3 构建高阶过程的抽象

前面已经见识过过程作为描述针对数字的组合操作的抽象，在构建时并不需要考虑具体数字的值。例如定义立方过程：

In [1]:
(define (cube x) (* x x x))

这里描述的是“怎样求立方”的方法，而不是具体数的立方值。

即使到现在为止遇到的仅仅只是处理数学运算的过程，如果我们构建的抽象局限于只能接收数字作为参数的过程，我们也能感觉到受到很大的限制。通常有一些相同的编程模式会多次出现在不同的过程中，为了表达这些过程我们需要构建能够接收其它过程作为参数或返回值的过程，称为**高阶过程（higher-order procedures）**。

### 1.3.1 过程作为参数

考虑下面三个过程：1. 计算整数a到b之间整数的和：

In [3]:
(define (sum-integers a b)
  (if (> a b)
      0
      (+ a (sum-integers (+ a 1) b) )))

2.整数a到b之间所有整数的立方和：

In [4]:
(define (cube x) (* x x x))
(define (sum-cubes a b)
  (if (> a b)
      0
      (+ (cube a)
         (sum-cubes (+ a 1) b))))

3.计算下面序列的和：（收敛至$\pi/8$）

$$\frac{1}{1 * 3} + \frac{1}{5 * 7} + \frac{1}{9 * 11} + \dots$$

In [5]:
(define (pi-sum a b)
  (if (> a b)
      0
      (+ (/ 1.0 (* a (+ a 2)))
         (pi-sum (+ a 4) b))))

上面三个过程的模式非常相似，差别仅在于计算相加项和计算下一个a的值的过程不同，由此我们可以得到上面三个过程的模板，只需要将不同的过程作为参数插入进模板中即可得到不同的计算过程：

In [6]:
(define (<name> a b)
  (if (> a b)
      0
      (+ (<term> a)
         (<name> (<next> a) b))))

实际上数学家们在很早之前就已经发明了这样的抽象用于描述序列的和：

$$\sum^b_{n=a}{f(n)} = f(a) + \dots + f(b)$$

这样的抽象表示让数学家们可以直接对求和的概念进行操作而不是针对某一特定求和的过程。我们也可以对上面提到的3个求和过程进行抽象：

In [7]:
(define (sum term a next b)
  (if (> a b)
      0
      (+ (term a)
         (sum term (next a) next b))))

然后不同的求和过程可以通过不同的参数设定给出：

In [9]:
(define (inc n) (+ n 1))
(define (sum-cubes a b)
  (sum cube a inc b))

(sum-cubes 1 10)

3025

In [10]:
(define (identity x) x)
(define (sum-integers a b)
  (sum identity a inc b))

(sum-integers 1 10)

55

In [11]:
(define (pi-sum a b)
  (define (pi-term x)
    (/ 1.0 (* x (+ x 2))))
  (define (pi-next x)
    (+ x 4))
  (sum pi-term a pi-next b))

(* 8 (pi-sum 1 1000))

3.139592655589783

有了对序列任意序列求和的抽象概念之后，我们也可以像数学家一样基于求和推出极限的概念：

$$\int_a^bf = [ f(a + \frac{dx}{2}) + f(a + dx + \frac{dx}{2}) + f(a + 2dx + \frac{dx}{2}) + \dots ] dx$$

In [13]:
(define (integral f a b dx)
  (define (add-dx x)
    (+ x dx))
  (* (sum f (+ a (/ dx 2.0)) add-dx b)
     dx))

(integral cube 0 1 0.01) ; 0到1之间立方函数的积分为1/4

0.24998750000000042

###1.3.2 用 Lambda 构建过程

`lambda` 是特殊形式。用于构建只会使用一次的过程，省去绑定过程名称的麻烦。上一节中的`pi-sum`可以利用`lambda`省去`pi-term`和`pi-next`这些过程：

In [1]:
(define (pi-sum a b)
  (sum (lambda (x) (/ 1.0 (* x (+ x 2))))
       a
       (lambda (x) (+ x 4))
       b))

`lambda`定义方式与`define`一样，只是不需要给过程指定名称。下面两个过程是一样的：

In [2]:
(define (plus4 x) (+ x 4))

(define plus4 (lambda (x) (+ x 4)))

**用 let 定义局部变量**

`lambda`的另一用途可以定义局部变量，例如计算下面的方程：

$$f(x,y) = x(1 + xy)^2 + y(1 - y) + (1 + xy)(1 - y)$$

也可以表示为：

$$\begin{align}a = & \ 1 + xy \\b = & \ 1 - y \\f(x,y) = & \ xa^2 + yb + ab\end{align}$$

In [7]:
(define (square x) (* x x))
(define (f x y)
  (define (f-helper a b)
    (+ (* x (square a)) (* y b) (* a b)))
  (f-helper (+ 1 (* x y)) (- 1 y)))

;; 通过lambda简化
(define (f x y)
  ((lambda (a b)
    (+ (* x (square a)) (* y b) (* a b)))
   (+ 1 (* x y))
   (- 1 y)))

4

`let`使这一过程更加简单：

In [8]:
(define (f x y)
  (let ((a (+ 1 (* x y)))
        (b (- 1 y)))
    (+ (* x (square a)) (* y b) (* a b))))

`let`语法（糖）如下：

```scheme
(let ((<var1> <exp1>)
      (<var2> <exp2>)
      ...)
  <body>)
```

等价于：

```scheme
((lambda (<var1>...<varn>) (<body>)
  <exp1>
  ...
  <expn>)
```

* `let`的作用范围仅限于`<body>`；

```scheme
(+ (let ((x 3))
      (+ x (* x 10)))
  x)
  
;; 若 x = 5，结果为38
```

* 变量赋值不属于`let`范围。

```scheme
(let ((x 3)
      (y (+ x 2)))
  (* x y))

;; 若 x = 2，结果为12
```

我们也可以用内部`define`完成相同的事，只是`define`更常用于定义过程，`let`通常用于定义局部变量。